In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt

from importlib import reload

import models
import plotting
import dataloaders as dl
import traintest as tt

Files already downloaded and verified
Files already downloaded and verified


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
import resnet
rs50 = resnet.ResNet50().to(device)

In [4]:
lr = .1

optimizer = optim.Adam( rs50.parameters(), lr=lr, weight_decay=0.005)
for epoch in range(50):
    tt.train(rs50, device, dl.CIFAR10_train_loader, optimizer, epoch)

optimizer.param_groups[0]['lr'] *= .1
for epoch in range(50):
    tt.train(rs50, device, dl.CIFAR10_train_loader, optimizer, epoch)

optimizer.param_groups[0]['lr'] *= .1
for epoch in range(50):
    tt.train(rs50, device, dl.CIFAR10_train_loader, optimizer, epoch)

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.346770
Train Epoch: 0 [10000/50000 (20%)]	Loss: 2.197913
Train Epoch: 0 [20000/50000 (40%)]	Loss: 2.409170
Train Epoch: 0 [30000/50000 (60%)]	Loss: 2.477228
Train Epoch: 0 [40000/50000 (80%)]	Loss: 2.494777
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.272688
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.691129
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.715252
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.342159
Train Epoch: 1 [40000/50000 (80%)]	Loss: 2.298324
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.390244
Train Epoch: 2 [10000/50000 (20%)]	Loss: 2.503430
Train Epoch: 2 [20000/50000 (40%)]	Loss: 2.269393
Train Epoch: 2 [30000/50000 (60%)]	Loss: 2.358509
Train Epoch: 2 [40000/50000 (80%)]	Loss: 2.325360
Train Epoch: 3 [0/50000 (0%)]	Loss: 2.338995
Train Epoch: 3 [10000/50000 (20%)]	Loss: 2.537717
Train Epoch: 3 [20000/50000 (40%)]	Loss: 2.830940
Train Epoch: 3 [30000/50000 (60%)]	Loss: 2.339609
Train Epoch: 3 [40000/50000 (80%)]	Loss: 2.312537
Train Epoch: 4 [0/50

Train Epoch: 33 [0/50000 (0%)]	Loss: 2.446251
Train Epoch: 33 [10000/50000 (20%)]	Loss: 2.596182
Train Epoch: 33 [20000/50000 (40%)]	Loss: 2.367115
Train Epoch: 33 [30000/50000 (60%)]	Loss: 2.406722
Train Epoch: 33 [40000/50000 (80%)]	Loss: 2.298795
Train Epoch: 34 [0/50000 (0%)]	Loss: 2.514759
Train Epoch: 34 [10000/50000 (20%)]	Loss: 2.266004
Train Epoch: 34 [20000/50000 (40%)]	Loss: 2.317819
Train Epoch: 34 [30000/50000 (60%)]	Loss: 2.313518
Train Epoch: 34 [40000/50000 (80%)]	Loss: 2.508703
Train Epoch: 35 [0/50000 (0%)]	Loss: 2.340260
Train Epoch: 35 [10000/50000 (20%)]	Loss: 2.420560
Train Epoch: 35 [20000/50000 (40%)]	Loss: 2.261135
Train Epoch: 35 [30000/50000 (60%)]	Loss: 2.409414
Train Epoch: 35 [40000/50000 (80%)]	Loss: 2.518813
Train Epoch: 36 [0/50000 (0%)]	Loss: 2.293327
Train Epoch: 36 [10000/50000 (20%)]	Loss: 2.339828
Train Epoch: 36 [20000/50000 (40%)]	Loss: 5.531427
Train Epoch: 36 [30000/50000 (60%)]	Loss: 3.133102
Train Epoch: 36 [40000/50000 (80%)]	Loss: 2.390771


Train Epoch: 16 [0/50000 (0%)]	Loss: 1.399311
Train Epoch: 16 [10000/50000 (20%)]	Loss: 1.270780
Train Epoch: 16 [20000/50000 (40%)]	Loss: 1.356835
Train Epoch: 16 [30000/50000 (60%)]	Loss: 1.414260
Train Epoch: 16 [40000/50000 (80%)]	Loss: 1.576173
Train Epoch: 17 [0/50000 (0%)]	Loss: 1.345813
Train Epoch: 17 [10000/50000 (20%)]	Loss: 1.404096
Train Epoch: 17 [20000/50000 (40%)]	Loss: 1.438952
Train Epoch: 17 [30000/50000 (60%)]	Loss: 1.360859
Train Epoch: 17 [40000/50000 (80%)]	Loss: 1.474809
Train Epoch: 18 [0/50000 (0%)]	Loss: 1.447723
Train Epoch: 18 [10000/50000 (20%)]	Loss: 1.317713
Train Epoch: 18 [20000/50000 (40%)]	Loss: 1.506468
Train Epoch: 18 [30000/50000 (60%)]	Loss: 1.396587
Train Epoch: 18 [40000/50000 (80%)]	Loss: 1.330742
Train Epoch: 19 [0/50000 (0%)]	Loss: 1.354545
Train Epoch: 19 [10000/50000 (20%)]	Loss: 1.329474
Train Epoch: 19 [20000/50000 (40%)]	Loss: 1.274204
Train Epoch: 19 [30000/50000 (60%)]	Loss: 1.448821
Train Epoch: 19 [40000/50000 (80%)]	Loss: 1.402253


Train Epoch: 48 [40000/50000 (80%)]	Loss: 1.322467
Train Epoch: 49 [0/50000 (0%)]	Loss: 1.240588
Train Epoch: 49 [10000/50000 (20%)]	Loss: 1.257221
Train Epoch: 49 [20000/50000 (40%)]	Loss: 1.202362
Train Epoch: 49 [30000/50000 (60%)]	Loss: 1.279704
Train Epoch: 49 [40000/50000 (80%)]	Loss: 1.247677
Train Epoch: 0 [0/50000 (0%)]	Loss: 1.422261
Train Epoch: 0 [10000/50000 (20%)]	Loss: 1.289177
Train Epoch: 0 [20000/50000 (40%)]	Loss: 1.170123
Train Epoch: 0 [30000/50000 (60%)]	Loss: 1.202801
Train Epoch: 0 [40000/50000 (80%)]	Loss: 1.150357
Train Epoch: 1 [0/50000 (0%)]	Loss: 1.039217
Train Epoch: 1 [10000/50000 (20%)]	Loss: 1.183859
Train Epoch: 1 [20000/50000 (40%)]	Loss: 1.060854
Train Epoch: 1 [30000/50000 (60%)]	Loss: 1.054697
Train Epoch: 1 [40000/50000 (80%)]	Loss: 1.099733
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.217858
Train Epoch: 2 [10000/50000 (20%)]	Loss: 1.330243
Train Epoch: 2 [20000/50000 (40%)]	Loss: 1.138868
Train Epoch: 2 [30000/50000 (60%)]	Loss: 1.135051
Train Epoch: 2

Train Epoch: 31 [40000/50000 (80%)]	Loss: 1.217609
Train Epoch: 32 [0/50000 (0%)]	Loss: 0.812598
Train Epoch: 32 [10000/50000 (20%)]	Loss: 1.021685
Train Epoch: 32 [20000/50000 (40%)]	Loss: 0.685759
Train Epoch: 32 [30000/50000 (60%)]	Loss: 0.941149
Train Epoch: 32 [40000/50000 (80%)]	Loss: 0.861844
Train Epoch: 33 [0/50000 (0%)]	Loss: 0.858044
Train Epoch: 33 [10000/50000 (20%)]	Loss: 0.890958
Train Epoch: 33 [20000/50000 (40%)]	Loss: 0.912687
Train Epoch: 33 [30000/50000 (60%)]	Loss: 1.070073
Train Epoch: 33 [40000/50000 (80%)]	Loss: 1.054520
Train Epoch: 34 [0/50000 (0%)]	Loss: 0.792254
Train Epoch: 34 [10000/50000 (20%)]	Loss: 1.028690
Train Epoch: 34 [20000/50000 (40%)]	Loss: 0.833599
Train Epoch: 34 [30000/50000 (60%)]	Loss: 1.058204
Train Epoch: 34 [40000/50000 (80%)]	Loss: 0.849653
Train Epoch: 35 [0/50000 (0%)]	Loss: 0.727045
Train Epoch: 35 [10000/50000 (20%)]	Loss: 0.863765
Train Epoch: 35 [20000/50000 (40%)]	Loss: 0.955480
Train Epoch: 35 [30000/50000 (60%)]	Loss: 0.825400


In [5]:
torch.save(rs50, 'SavedModels/resnet50.pth')

In [9]:
rs50 = torch.load('SavedModels/resnet50.pth')
gmm = models.GMM(1000, 3072)
gmm.find_solution(dl.X_CIFAR10, initialize=True, iterate=False, use_kmeans=False)
gmm_model = models.RobustModel(rs50, gmm, -7., dim=3072).to(device)
torch.save(gmm_model_CIFAR10, 'SavedModels/gmm_CIFAR10.pth')

NameError: name 'gmm_model_CIFAR10' is not defined

In [7]:
reload(models)

<module 'models' from '/home/alexm/project/notebooks/gmm-robust/models.py'>

In [ ]:
gmm_model.loglam.data = torch.tensor(-7., device=device)

data = enumerate(dl.CIFAR10_test_loader).__next__()[1][0]
y = rs50(data.to(device))
y2 = gmm_model(data.to(device))

plotting.plot_samples([y, y2], data, dataset = 'CIFAR10')

In [ ]:
import matplotlib.pyplot as plt
data = torch.rand(10,3,32,32)
y = rs50(data.to(device))
y2 = gmm_model(data.to(device))

plotting.plot_samples([y, y2], data, dataset = 'CIFAR10')

In [ ]:
min_conf = .1
tt.test(gmm_model, device, dl.CIFAR10_test_loader, min_conf=min_conf)
tt.test(rs50, device, dl.CIFAR10_test_loader, min_conf=min_conf)

tt.test_adv(gmm_model, device, CIFAR100_test_loader, min_conf=min_conf)
tt.test_adv(rs50, device, CIFAR100_test_loader, min_conf=min_conf)

In [ ]:
from torchvision import datasets, transforms

CIFAR100_test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR100('../data', train=False, transform=dl.transform, download=True),
        batch_size=100, shuffle=False)

In [ ]:
#gmm_model.loglam.data = torch.tensor(-3., device=device)

data = enumerate(CIFAR100_test_loader).__next__()[1][0]
y = rs50(data.to(device))
y2 = gmm_model(data.to(device))

plotting.plot_samples([y, y2], data, dataset = 'CIFAR10')

In [ ]:
def bound(L, model):
    xi = (model.mm.calculate_bound(L)-model.loglam).exp()
    return .1*(1+10*xi)/(1+xi)


In [ ]:
bound(15., gmm_model)

In [ ]:
tt.test(rs50, device, dl.CIFAR10_train_loader, min_conf=min_conf)